# Reload PSD Data & Compare HVSR Results

This notebook analyzes seismic data by calculating and comparing Horizontal to Vertical Spectral Ratios (HVSR) from Power Spectral Density (PSD) dat This notebook provides a comprehensive workflow for analyzing site characteristics using HVSR, particularly useful for identifying resonance frequencies and their temporal variations.a.

# Import Required Libraries

Sets up the Python environment with:
- Basic utilities: datetime, os, glob
- Data processing: numpy, pandas
- Visualization: matplotlib (configured for Adobe Illustrator compatibility)
- Seismic processing: 
  - ObsPy components (UTCDateTime, read, read_inventory, PPSD)
  - FDSN client for data access
- Progress tracking: tqdm
- Custom utilities: seismosocialdistancing module

In [ ]:
import datetime
import os
from glob import glob

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42  # to edit text in Illustrator
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe
import numpy as np
import pandas as pd
import tqdm
import warnings

from obspy import UTCDateTime, read, read_inventory
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNNoDataException
from obspy.signal import PPSD
from msnoise.api import psd_ppsd_to_dataframe

%load_ext autoreload
%autoreload 2
import seismosocialdistancing

# Configuration Parameters

Sets up analysis parameters:
- Time period: 2024-01-01 to 2024-01-31
- Station details:
  - Network: 8N
  - Station: HB04
  - Location: 00
  - Channel: EHZ (vertical component)
- Analysis settings:
  - Dataset name: "example"
  - Timezone: Europe/Brussels
  - Site location: Grenoble (FR)
- Event markers defined for key dates

In [ ]:
# Make sure you take at least a full week (>=7 days) before the first "ban"
start = UTCDateTime("2024-01-01")
# Leaving UTCDateTime() empty means "now":
end = UTCDateTime("2024-02-01")

network = "8N"
station = "HB0*"
location = "00"
channel = "EH*"
dataset = "hautbois-grenoble"
time_zone = "Europe/Brussels"
sitedesc = "in Grenoble (FR)"

logo = None # 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/44/Logo_SED_2014.png/220px-Logo_SED_2014.png'
bans = {"2024-01-01 00:00":'New Year', 
        "2024-01-15 00:00":'Half',
        "2024-01-31 23:59": "End of Record"}

datelist = pd.date_range(start.datetime, min(end, UTCDateTime()).datetime, freq="D")

# Load Power Spectral Density Data

Iterates through date range to load pre-computed PSD data:
- Reads NPZ files for each day
- Organizes data by SEED ID (station identifier)
- Handles warnings during data loading
- Combines multiple days of data into single PPSD objects

In [ ]:
ppsds = {}
pbar = tqdm.tqdm(datelist)
for day in pbar:
    datestr = "%03i"%int(day.strftime("%j"))
    fn_pattern = os.path.join("seismorms", "{}_{}_*.npz".format(dataset, datestr))
    pbar.set_description("Reading %s" % fn_pattern)
    for fn in glob(fn_pattern):
        mseedid = fn.replace(".npz", "").split("_")[-1]
        if mseedid not in ppsds:
            ppsds[mseedid] = PPSD.load_npz(fn)#, allow_pickle=True)
        else:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                ppsds[mseedid].add_npz(fn)#, allow_pickle=True)

# Velocity Conversion Utilities

Two key functions for converting acceleration PSDs to velocity spectra:

1. convert_to_velocity(df):
   Converts acceleration power (in dB) to velocity amplitude using these steps:
   
   a) First, acceleration power spectral density to acceleration amplitude:
      $A(f) = \sqrt{10^{PSD/10}}$
   
   b) Then, acceleration to velocity conversion in frequency domain:
      $V(f) = \frac{A(f)}{2\pi f}$
   
   Combining both steps into a single equation:
   $V(f) = \sqrt{\frac{10^{PSD/10}}{(2\pi f)^2}}$
   
   Where:
   - $PSD$: Power Spectral Density in dB (acceleration)
   - $f$: frequency in Hz
   - $A(f)$: acceleration amplitude
   - $V(f)$:qrt(10.0 ** (df / 10.) / w2f ** 2).sqrt(10.0 ** (df / 10.) / w2f ** 2)
   - Applies the conversionsformation

In [ ]:
def convert_to_velocity(df):
    df = df.resample("30Min").mean()
    df.columns = 1. / df.columns
    df = df.sort_index(axis=1)
    df = df.dropna(axis=1, how='all')

    w2f = (2.0 * np.pi * df.columns)
    # The acceleration amplitude spectrum and velocity spectral amplitude (not power)
    vamp = np.sqrt(10.0 ** (df / 10.) / w2f ** 2)
    return vamp

def psd_to_vel(seed_id):
    Z = psd_ppsd_to_dataframe(ppsds[seed_id])
    Z = Z.dropna(axis=1, how="all")
    Z = convert_to_velocity(Z)    
    return Z

# Process Component Data

Converts PSD data to velocity spectra for:
- Vertical component (Z)
- North-South component (N)
- East-West componen out]

In [ ]:
def get_spectra_and_hvsr(netstaloc):
    Z = psd_to_vel(f"{netstaloc}.EHZ")
    N = psd_to_vel(f"{netstaloc}.EHN")
    E = psd_to_vel(f"{netstaloc}.EHE")
    HVSR = (N+E)/(2*Z)
    return Z, N, E, HVSR

# Spectral Analysis and HVSR Calculation

Creates two visualization plots:
1. Average spectrum per component:
   - Shows Z and N components
   - Marks 0.27 Hz reference line
   - Includes legend and scaling

2. HVSR from average spectra:
   - Calculates N/Z ratio
   - Shows frequency re- onse
   - Marks resonance frequency

In [ ]:
netstaloc = "8N.HB01.00"
Z, N, E, HVSR = get_spectra_and_hvsr(netstaloc)
Zm = Z.mean()
Nm = N.mean()
Em = E.mean()
HVSR = HVSR.mean()
HVSR_from_mean = (Nm+Em)/(2*Zm)

plt.semilogx(Zm.index, Zm, label="Z")
plt.semilogx(Nm.index, Nm, label="N")
plt.semilogx(Em.index, Em, label="E")
plt.axvline(0.27, c='r', ls="--")
plt.legend()
plt.title("Average Spectrum per component")
plt.ylim(0, np.percentile(Zm, 99.5))
plt.show()

plt.figure()
plt.semilogx(HVSR.index, HVSR, label="HVSR (PSD, mean of HVSR windows)")
plt.semilogx(HVSR_from_mean.index, HVSR_from_mean, label="HVSR (PSD, HVSR of mean Z,N,E)")
plt.axvline(0.27, c='r', ls="--")
plt.legend()
plt.title("HVSR from the ratio of average spectra")
plt.show()

# Compare with Geopsy

We will now load the precomputed average HVSR computed in Geopsy : $HVSR_{geopsy} = mean(HVSR_w)$ for each w window and compare it to the HVSR we obtain from the PSDs computation: either the mean of HVSR windows, or the HVSR from mean spectra.

In [ ]:
def read_HV(HV_file):
    """
    Read the .hv output files of Geopsy exported from the HV module.
    The definition returns a pandas series of: 
    Freq, A, A_min, A_max 
    with 
    Freq: H/V geometrically averaged over all individual H/V curves (windows)
    A0: the H/V amplitude curve
    A_min: The H/V amplitude minimum standard deviation.
    A_max: The H/V amplitude maximum standard deviation.
    """
    df = pd.read_csv(HV_file, delimiter='\t',names=['Frequency', 'Average', 'Min', 'Max'], comment='#')
    Freq = df["Frequency"]
    A_min = df['Min']
    A = df['Average']
    A_max = df['Max']
    NaNs = np.isnan(df)
    df[NaNs] = 0
    return Freq, A, A_min, A_max

def plot_HV(Freq, A, A_min, A_max,):
    
    """
    Plot the HV curve from an .hv log file using all necessary params:
    Freq, A, A_min, A_max, Amax_f0, Fmax_f0, Amin, Amax, label, color
    Label: fill in whatever you want
    color: color the H/V curve
    Other params see get_params_from_HV() and get_params_from_HV_curve()
    """
    
    plt.semilogx(Freq, A, label='HVSR Geopsy (mean of HVSR windows)', zorder = 0)
    plt.fill_between(Freq, A_min,A_max,facecolor='lightblue', edgecolor="none", alpha=0.5, label = 'error')

In [ ]:
netstaloc = "8N.HB04.00"

# Geopsy output - 2 years!! of data
Freq, A, A_min, A_max = read_HV(f"DATA/HVSR_AVG/{netstaloc}_HVSR_mean.hv")

# This HVSR:
Z, N, E, HVSR = get_spectra_and_hvsr(netstaloc)
Zm = Z.mean()
Nm = N.mean()
Em = E.mean()
HVSR_PSD = HVSR.mean()
HVSR_PSD_std = HVSR.std()
HVSR_from_mean = (Nm+Em)/(2*Zm)

plt.figure(figsize=(16,8))
plot_HV(Freq, A, A_min, A_max)

plt.semilogx(HVSR_PSD.index, HVSR_PSD, label="HVSR PSD (mean of HVSR windows)")
plt.fill_between(HVSR_PSD.index, HVSR_PSD-HVSR_PSD_std, HVSR_PSD+HVSR_PSD_std, label="HVSR PSD (STD of HVSR windows)",
                 zorder=-1, facecolor="orange", edgecolor="none", alpha=0.5)
plt.semilogx(HVSR_from_mean.index, HVSR_from_mean, label="HVSR PSD (HVSR of mean Z,N,E)")

plt.axvline(0.27, c='r', ls="--")
plt.legend()
plt.grid(which='both')
plt.title(f"HVSR from the ratio of average spectra for {netstaloc}")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.show()

# Continuous HVSR Processing

Creates time-frequency visualizations:
1. Full spectrum HVSR:
   - Shows ratio evolution over time
   - Uses logarithmic frequency scale
   - Includes colorbar for amplitude

2. Focused low-frequency analysis:
   - Zooms on main resonance peak
   - Frequency range: 0.2-0.35 Hz
   - Shows temporal variations in site response

In [ ]:
netstaloc = "8N.HB04.00"

In [ ]:
Z, N, E, HVSR = get_spectra_and_hvsr(netstaloc)
HVSR.head()

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1,1, figsize=(16,7), sharex=True)

data_f = HVSR.copy()
data_f = data_f.dropna()
vmin, vmax = np.percentile(data_f.values, [0.5,99.5])
plt.pcolormesh(data_f.index, data_f.columns, data_f.T, cmap="viridis",
                       vmin=vmin, vmax=vmax, rasterized=True)
plt.colorbar(shrink=0.7).set_label("Amplitude (dB)")
# plt.ylim(0.05,20)
plt.yscale("log")
plt.ylabel("Frequency (Hz)")

minx, maxx = plt.xlim()
plt.axhline(0.27, c='r', ls="--")
fig.autofmt_xdate()

plt.tight_layout()


### Zoom on the main low frequency peak

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1,1, figsize=(16,7), sharex=True)

data_f = HVSR.copy()
data_f = data_f.dropna()
# data_f = data_f.rolling("6H", center=True).mean()
vmin, vmax = np.percentile(data_f.loc[:,0.2:0.35].values, [0.05,99.5])
plt.pcolormesh(data_f.index, data_f.columns, data_f.T, cmap="viridis",
                       vmin=vmin, vmax=vmax, rasterized=True)
plt.colorbar(shrink=0.7).set_label("Amplitude (dB)")
# plt.ylim(0.05,20)
# plt.yscale("log")
plt.ylabel("Frequency (Hz)")

minx, maxx = plt.xlim()
plt.axhline(0.27, c='r', ls="--")
fig.autofmt_xdate()
plt.ylim(0.2, 0.35)
plt.tight_layout()


## Cross-Station Spectral Ratios

In [ ]:
for channel in ["EHE", "EHN", "EHZ"]:
    HB01 = psd_to_vel(f"8N.HB01.00.{channel}").mean().loc[0.05:95]
    HB04 = psd_to_vel(f"8N.HB04.00.{channel}").mean().loc[0.05:95]
    fig, axes = plt.subplots(2,1, sharex=True, figsize=(12,8))
    plt.sca(axes[0])
    plt.semilogx(HB01.index, HB01, label="HB01 (top)")
    plt.semilogx(HB04.index, HB04, label="HB04 (bottom)")
    plt.axvline(0.27, c='r', ls="--")
    plt.legend()
    plt.title(f"Average Spectrum: {channel}")
    # plt.ylim(0, np.percentile(Zm, 99.9))
    plt.grid(which="both")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")

    plt.sca(axes[1])    
    RATIO = HB01/HB04
    plt.semilogx(RATIO.index, RATIO, label="HB01/HB04 (top/bottom)", c="g")
    plt.axvline(0.27, c='r', ls="--")
    plt.legend()
    plt.title(f"Amplification: {channel}")
    plt.grid(which="both")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    plt.show()